In [1]:
import pandas as pd
import numpy as np
import re
import nltk
import jieba

In [87]:
with open('../txt/sentiment_XS_30k.txt',encoding='utf8',mode='r') as f:
    wblist=[]
    for s in f.readlines():
        wblist.append(s.strip())

In [88]:
wblist

['labels,text',
 'positive,奖励 就是 亲',
 'positive,谢谢 妹妹 加 朋友',
 'positive,喜欢 吃 南浮',
 'positive,好 吧 小 很 强悍 怕 不',
 'negative,他 女朋友 旁边 所以 不 方便 跟 说话',
 'positive,讲 的话 真 幽默',
 'negative,真的 那 好 心疼',
 'positive,无所不能 可是 刚才 差点 要 上 百度',
 'positive,就是 漂亮 嘛 完美',
 'negative,跟 一个 低级 计算机系统 说话 要 那么 讲究',
 'negative,不行 必须 起',
 'positive,这么 丑 还 漂亮',
 'negative,一点 也 不好 没有 人 懂 知道',
 'positive,噢 咋 设置 美女',
 'positive,石家庄 五星级 酒店',
 'positive,机器人 怎么 可能 忙',
 'positive,哪有 这是 微笑 微笑',
 'negative,无聊 阿肿 办',
 'negative,无语 看来 白说',
 'negative,快点 别 废话 多',
 'negative,不好 咋 地',
 'positive,我要 看 美女视频',
 'negative,讨厌 手机信号 不好',
 'negative,吃饭 不行 哪',
 'positive,喜欢 夕颜 咋办',
 'negative,低 哦 肆意 眼',
 'negative,作为 一个 强攻 失败',
 'negative,小孩 上网 有害 健康',
 'negative,心情 不好 可以 干嘛',
 'negative,突然 有种 莫名 心疼',
 'negative,为什么 大多数 有钱人 都 无情',
 'negative,呵呵 真 不好意思 看来 对 没有 用 只是 别人 工具',
 'negative,啥 不好 说 说',
 'positive,高中物理 要 怎么 学',
 'positive,陪 我哥 好 好玩 会',
 'positive,最好 专业 什么',
 'positive,叫 依依 还是 叫小琳',
 'negative,又 错乱 哦',
 'positive,翻译 希望 能 一起',
 'positi

In [3]:
sentment_table = pd.read_excel('../dict/情感词典修改版.xlsx')

In [18]:
sentment_table.drop(['Unnamed: 10','Unnamed: 11'],inplace=True,axis=1)

In [20]:
sentment_table.head()

,詞語,詞性種類,詞義數,詞義序號,情感分類,強度,極性,輔助情感分類,強度2,極性2
0,髒亂,adj,1.0,1.0,NN,7,2,NaN,NaN,NaN
1,糟報,adj,1.0,1.0,NN,5,2,NaN,NaN,NaN
2,早衰,adj,1.0,1.0,NE,5,2,NaN,NaN,NaN
3,責備,verb,1.0,1.0,NN,5,2,NaN,NaN,NaN
4,賊眼,noun,1.0,1.0,NN,5,2,NaN,NaN,NaN


In [32]:
pos_table = pd.read_excel('../dict/情感词典修改版.xlsx',sheet_name='Sheet2')
pos_table.head()

,posword,score
0,清瑩,5
1,輕倩,5
2,晴麗,5
3,求索,3
4,熱潮,5


In [37]:
pos_dict = dict(zip(list(pos_table.posword),list(pos_table.score)))
pos_dict

{'清瑩': 5,
 '輕倩': 5,
 '晴麗': 5,
 '求索': 3,
 '熱潮': 5,
 '仁政': 5,
 '榮名': 5,
 '柔膩': 5,
 '瑞雪': 5,
 '擅名': 7,
 '神采': 5,
 '省便': 5,
 '盛開': 5,
 '盛譽': 5,
 '詩仙': 7,
 '爽脆': 5,
 '素潔': 3,
 '甜頭': 5,
 '頭名': 5,
 '婉妙': 3,
 '溫淑': 1,
 '穩厚': 3,
 '沃壤': 5,
 '無上': 5,
 '喜人': 5,
 '先賢': 5,
 '鮮妍': 5,
 '相稱': 5,
 '驍騎': 3,
 '新巧': 5,
 '新意': 5,
 '信奉': 5,
 '急智': 3,
 '技藝': 3,
 '雄勁': 3,
 '雄肆': 3,
 '秀俊': 5,
 '煦煦': 5,
 '勳績': 5,
 '雅淡': 5,
 '雅興': 5,
 '嚴整': 5,
 '泱泱': 5,
 '怡悅': 5,
 '義診': 5,
 '英發': 5,
 '英偉': 5,
 '瑩潤': 3,
 '勇健': 3,
 '優厚': 5,
 '幽雅': 5,
 '有識': 5,
 '友鄰': 5,
 '玉人': 5,
 '元勳': 5,
 '佳句': 7,
 '佳趣': 5,
 '兼愛': 3,
 '堅守': 7,
 '簡雅': 5,
 '薦舉': 7,
 '見稱': 7,
 '見聞': 1,
 '獎旗': 3,
 '獎許': 7,
 '姣麗': 5,
 '嬌麗': 5,
 '驕子': 5,
 '解惑': 3,
 '津要': 5,
 '金嗓子': 3,
 '勻淨': 3,
 '贊詞': 3,
 '真純': 3,
 '箴言': 5,
 '珍玩': 7,
 '正理': 5,
 '指導': 3,
 '至理': 7,
 '致敬': 5,
 '智多星': 5,
 '忠魂': 7,
 '主力軍': 5,
 '準繩': 5,
 '卓特': 7,
 '進益': 5,
 '奏凱': 5,
 '尊服': 3,
 '盡瘁': 5,
 '精白': 3,
 '精潔': 7,
 '精妙': 5,
 '精髓': 7,
 '精要': 3,
 '晶潔': 5,
 '景觀': 5,
 '敬信': 7,
 '救亡': 1,
 '據實': 1,
 '眷懷': 

In [13]:
neg_table = pd.read_excel('../dict/情感词典修改版.xlsx',sheet_name='Sheet3')
neg_table.head()

,negword,score
0,髒亂,7
1,糟報,5
2,早衰,5
3,責備,5
4,賊眼,5


In [42]:
neg_dict = dict(zip(list(neg_table.negword),map(lambda a:a*(0-1),list(neg_table.score)) ))
neg_dict

{'髒亂': -7,
 '糟報': -5,
 '早衰': -5,
 '責備': -5,
 '賊眼': -5,
 '戰禍': -5,
 '招災': -5,
 '折辱': -5,
 '中山狼': -5,
 '譸張': -5,
 '株連': -7,
 '墜毀': -5,
 '滋生': -5,
 '自恃': -5,
 '走後門': -5,
 '阻力': -3,
 '罪狀': -9,
 '做戲': -7,
 '哀鴻遍野': -7,
 '肮裡骯髒': -5,
 '不成話': -5,
 '昭然若揭': -9,
 '有口無心': -9,
 '不乾不淨': -3,
 '手腳不乾淨': -9,
 '索然寡味': -9,
 '死板板': -9,
 '松垮垮': -9,
 '危在旦夕': -9,
 '無紀律': -9,
 '無實質': -9,
 '險峭': -1,
 '不明不白': -5,
 '不清不白': -7,
 '不透明': -3,
 '不雅': -3,
 '不足為外人道': -3,
 '玄妙': -9,
 '罪惡多端': -9,
 '皺巴': -9,
 '逞性子': -7,
 '臭名昭彰': -9,
 '哀梨蒸食': -7,
 '度量小': -7,
 '翻雲覆雨': -5,
 '腐化墮落': -9,
 '高不成低不就': -3,
 '孤單單': -5,
 '寡斷': -7,
 '乖剌': -7,
 '含糊不清': -5,
 '毫不客氣': -5,
 '鬍子拉碴': -3,
 '荒誕派': -7,
 '荒淫無度': -9,
 '晦暝': -7,
 '昏黑': -3,
 '抱疚': -3,
 '憋著一肚子氣': -7,
 '愁慘': -5,
 '錯愕': -7,
 '掉魂': -7,
 '發牢騷': -3,
 '煩神': -5,
 '忿忿不平': -5,
 '惶然不安': -5,
 '急死': -7,
 '口誅筆伐': -5,
 '罵得狗血噴頭': -7,
 '人身攻擊': -5,
 '尖嘴薄舌': -5,
 '心如鐵石': -7,
 '見風轉舵': -7,
 '可駭': -9,
 '枯寂': -5,
 '狂恣': -9,
 '老辣': -5,
 '肋脦': -7,
 '哩哩羅羅': -3,
 '令人厭倦': -7,
 '碌碌無為': -7,
 '馬馬虎虎': -5,
 '莽莽撞撞':

In [46]:
sentment_dict={**pos_dict,**neg_dict}
sentment_dict

{'清瑩': 5,
 '輕倩': 5,
 '晴麗': 5,
 '求索': 3,
 '熱潮': 5,
 '仁政': 5,
 '榮名': 5,
 '柔膩': 5,
 '瑞雪': 5,
 '擅名': 7,
 '神采': 5,
 '省便': 5,
 '盛開': 5,
 '盛譽': 5,
 '詩仙': 7,
 '爽脆': 5,
 '素潔': 3,
 '甜頭': 5,
 '頭名': 5,
 '婉妙': 3,
 '溫淑': 1,
 '穩厚': 3,
 '沃壤': 5,
 '無上': 5,
 '喜人': 5,
 '先賢': 5,
 '鮮妍': 5,
 '相稱': 5,
 '驍騎': 3,
 '新巧': 5,
 '新意': 5,
 '信奉': 5,
 '急智': 3,
 '技藝': 3,
 '雄勁': 3,
 '雄肆': 3,
 '秀俊': 5,
 '煦煦': 5,
 '勳績': 5,
 '雅淡': 5,
 '雅興': 5,
 '嚴整': 5,
 '泱泱': 5,
 '怡悅': 5,
 '義診': 5,
 '英發': 5,
 '英偉': 5,
 '瑩潤': 3,
 '勇健': 3,
 '優厚': 5,
 '幽雅': 5,
 '有識': 5,
 '友鄰': 5,
 '玉人': 5,
 '元勳': 5,
 '佳句': 7,
 '佳趣': 5,
 '兼愛': 3,
 '堅守': 7,
 '簡雅': 5,
 '薦舉': 7,
 '見稱': 7,
 '見聞': 1,
 '獎旗': 3,
 '獎許': 7,
 '姣麗': 5,
 '嬌麗': 5,
 '驕子': 5,
 '解惑': 3,
 '津要': 5,
 '金嗓子': 3,
 '勻淨': 3,
 '贊詞': 3,
 '真純': 3,
 '箴言': 5,
 '珍玩': 7,
 '正理': 5,
 '指導': 3,
 '至理': 7,
 '致敬': 5,
 '智多星': 5,
 '忠魂': 7,
 '主力軍': 5,
 '準繩': 5,
 '卓特': 7,
 '進益': 5,
 '奏凱': 5,
 '尊服': 3,
 '盡瘁': 5,
 '精白': 3,
 '精潔': 7,
 '精妙': 5,
 '精髓': 7,
 '精要': 3,
 '晶潔': 5,
 '景觀': 5,
 '敬信': 7,
 '救亡': 1,
 '據實': 1,
 '眷懷': 

In [57]:
for w in sentment_dict.keys():
    jieba.suggest_freq(w,True)

In [83]:
sentment_table

,詞語,詞性種類,詞義數,詞義序號,情感分類,強度,極性,輔助情感分類,強度2,極性2
0,髒亂,adj,1.0,1.0,NN,7,2,NaN,NaN,NaN
1,糟報,adj,1.0,1.0,NN,5,2,NaN,NaN,NaN
2,早衰,adj,1.0,1.0,NE,5,2,NaN,NaN,NaN
3,責備,verb,1.0,1.0,NN,5,2,NaN,NaN,NaN
4,賊眼,noun,1.0,1.0,NN,5,2,NaN,NaN,NaN
5,戰禍,noun,1.0,1.0,ND,5,2,NC,5.0,2.0
6,招災,adj,1.0,1.0,NN,5,2,NaN,NaN,NaN
7,折辱,noun,1.0,1.0,NE,5,2,NN,5.0,2.0
8,中山狼,noun,1.0,1.0,NN,5,2,NaN,NaN,NaN
9,清峻,adj,1.0,1.0,PH,5,0,NaN,NaN,NaN


In [58]:
stop_words = [re.findall(r'\S+',w)[0] for w in open('../txt/stopwords_ch.txt',encoding='utf8').readlines() if len(re.findall(r'\S+',w))>0]
def sent2word(sentence,stop_words=stop_words):
    words = jieba.cut(sentence, HMM=False)
    words = [w for w in words if w not in stop_words]
    return words

In [59]:
stop_words

[',',
 '?',
 '、',
 '。',
 '“',
 '”',
 '《',
 '》',
 '！',
 '，',
 '：',
 '；',
 '？',
 '人民',
 '末##末',
 '啊',
 '阿',
 '哎',
 '哎呀',
 '哎喲',
 '唉',
 '我',
 '我們',
 '按',
 '按照',
 '依照',
 '吧',
 '吧噠',
 '把',
 '罷了',
 '被',
 '本',
 '本著',
 '比',
 '比方',
 '比如',
 '鄙人',
 '彼',
 '彼此',
 '邊',
 '別',
 '別的',
 '別說',
 '並',
 '並且',
 '不比',
 '不成',
 '不單',
 '不但',
 '不獨',
 '不管',
 '不光',
 '不過',
 '不僅',
 '不拘',
 '不論',
 '不怕',
 '不然',
 '不如',
 '不特',
 '不惟',
 '不問',
 '不只',
 '朝',
 '朝著',
 '趁',
 '趁著',
 '乘',
 '沖',
 '除',
 '除此之外',
 '除非',
 '除了',
 '此',
 '此間',
 '此外',
 '從',
 '從而',
 '打',
 '待',
 '但',
 '但是',
 '當',
 '當著',
 '到',
 '得',
 '的',
 '的話',
 '等',
 '等等',
 '地',
 '第',
 '叮咚',
 '對',
 '對於',
 '多',
 '多少',
 '而',
 '而況',
 '而且',
 '而是',
 '而外',
 '而言',
 '而已',
 '爾後',
 '反過來',
 '反過來說',
 '反之',
 '非但',
 '非徒',
 '否則',
 '嘎',
 '嘎登',
 '該',
 '趕',
 '個',
 '各',
 '各個',
 '各位',
 '各種',
 '各自',
 '給',
 '根據',
 '跟',
 '故',
 '故此',
 '固然',
 '關於',
 '管',
 '歸',
 '果然',
 '果真',
 '過',
 '哈',
 '哈哈',
 '呵',
 '和',
 '何',
 '何處',
 '何況',
 '何時',
 '嘿',
 '哼',
 '哼唷',
 '呼哧',
 '乎',
 '嘩',
 '還是',
 '還有',
 '換句話說',
 '換言之',


In [72]:
sent = '丁丁很有禮貌'

In [73]:
tokens = sent2word(sent)
tokens

['丁丁', '有禮貌']

In [76]:
def get_sentment(sent):
    tokens = sent2word(sent)
    score = 0
    countword = 0
    for w in tokens:
        
        if w in sentment_dict.keys():
            print((w,sentment_dict[w]))
            score += sentment_dict[w]
            countword += 1
    if countword != 0:
        return score/countword
    else:
        return 0

In [82]:
sent ="""



"""

get_sentment(sent)

0